In [3]:
import asyncio
from autogen_agentchat.agents import AssistantAgent,UserProxyAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

load_dotenv()


True

In [2]:
api_key = os.getenv('GOOGLE_API_KEY')
model_client = OpenAIChatCompletionClient(
    model='gemini-2.5-flash',
    api_key=api_key
)

In [4]:
writer = AssistantAgent(
    name="Writer",
    description="A writer agent that generates text based on user input.",
    model_client=model_client,
    system_message="You are a creative writer. Please write a story based on the user's input.",
)

reviewer = AssistantAgent(
    name="Reviewer",
    description="A reviewer agent that provides feedback on the text generated by the writer.",
    model_client=model_client,
    system_message="You are a reviewer. Please provide feedback on the text generated by the writer.",
)

In [5]:
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow

builder = DiGraphBuilder()
builder.add_node(writer)
builder.add_node(reviewer)
builder.add_edge(writer,reviewer)

graph = builder.build()

In [6]:
graph

DiGraph(nodes={'Writer': DiGraphNode(name='Writer', edges=[DiGraphEdge(target='Reviewer', condition=None, condition_function=None, activation_group='Reviewer', activation_condition='all')], activation='all'), 'Reviewer': DiGraphNode(name='Reviewer', edges=[], activation='all')}, default_start_node=None)

In [7]:
team = GraphFlow([writer,reviewer],graph=graph)

In [10]:
from autogen_agentchat.ui import Console
stream = team.run_stream(task ="Write a good poem about India in less than 30 words.")

await Console(stream)

---------- TextMessage (user) ----------
Write a good poem about India in less than 30 words.


---------- TextMessage (Writer) ----------
Vibrant hues, a timeless art,
Ancient wisdom, in every heart.
Sacred rivers, mountains tall,
India's spirit beckons all.
---------- TextMessage (Reviewer) ----------
This is another excellent poem!

Here's a breakdown of why it works so well:

*   **Conciseness:** At 21 words, it comfortably meets the "less than 30 words" requirement while packing a lot of meaning.
*   **Rich Imagery:** Phrases like "Vibrant hues," "timeless art," "Sacred rivers," and "mountains tall" paint a vivid and evocative picture of India.
*   **Thematic Depth:** It successfully touches upon several key aspects of India: its artistic heritage, deep-rooted wisdom, diverse geography, and welcoming spiritual essence.
*   **Flow and Rhyme:** The AABB rhyme scheme (art/heart, tall/all) gives it a pleasing musicality and makes it feel complete.
*   **Impact:** It effectively conveys a sense of India's allure and diverse beauty in a very compact form.

Great job once again!


TaskResult(messages=[TextMessage(id='d2e013bc-066c-45b2-b3ef-70946a63e4b2', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 21, 12, 2, 55, 250117, tzinfo=datetime.timezone.utc), content='Write a good poem about India in less than 30 words.', type='TextMessage'), TextMessage(id='0dfb6f80-1c35-41f0-b8a1-f4edcedabd1c', source='Writer', models_usage=RequestUsage(prompt_tokens=271, completion_tokens=33), metadata={}, created_at=datetime.datetime(2025, 8, 21, 12, 3, 2, 381969, tzinfo=datetime.timezone.utc), content="Vibrant hues, a timeless art,\nAncient wisdom, in every heart.\nSacred rivers, mountains tall,\nIndia's spirit beckons all.", type='TextMessage'), TextMessage(id='3926be15-ae96-425e-950e-3cb1a9a51c1b', source='Reviewer', models_usage=RequestUsage(prompt_tokens=303, completion_tokens=193), metadata={}, created_at=datetime.datetime(2025, 8, 21, 12, 3, 5, 750040, tzinfo=datetime.timezone.utc), content='This is another excellent poem!\n\nHere\'s a

In [17]:
import json
state = await team.save_state()

In [18]:
with open('state.json','w') as f:
    json.dump(state,f)